# Modelos de Ensemble

Una de las últimas tendencias en la modelización de inteligencia artificial se puede resumir como "conocimiento del todo o de la multitud". Lo que esta frase algo familiar define es el uso de una multitud de llamados "modelos débiles" en un meta-clasificador. El objetivo es generar un "modelo fuerte" basado en el conocimiento extraído por los "modelos débiles". Por ejemplo, aunque se detallará más adelante, en un Bosque Aleatorio se desarrollan múltiples Árboles de Decisión, mucho más simples. La combinación de estos en el Bosque Aleatorio supera el rendimiento de cualquiera de los modelos individuales. Los modelos que emergen de esta manera, como meta-clasificadores o meta-regresores, se denominan genéricamente **Modelos de Ensamble**.

Cabe mencionar que estos modelos no están limitados únicamente a árboles de decisión, sino que pueden estar compuestos por cualquier tipo de modelo de aprendizaje automático que se haya visto previamente. Incluso pueden ser modelos mixtos en los que no todos los modelos se han obtenido de la misma manera, sino que pueden ser creados a través del uso combinado de varias técnicas como K-NN, SVM, etc. Así, el primer criterio para clasificar los modelos de ensamble sería si son modelos homogéneos o heterogéneos. Sin embargo, este no es el único criterio para clasificar los modelos de ensamble; en esta unidad exploraremos varias formas de generar los modelos y cómo combinarlos posteriormente. También analizaremos más de cerca dos de las técnicas más comunes dentro de los modelos de ensamble, como el Bosque Aleatorio y _XGBoost_.

## Preparación de los datos

A diferencia de otros tutoriales, donde se ha utilizado el problema de la flor de iris como referencia, en este tutorial utilizaremos uno diferente. El problema también está incluido en el repositorio de UCI; aunque es pequeño, el número de variables aumenta significativamente, lo que nos dará un margen mayor para explorar. Específicamente, se trata de un problema clásico de aprendizaje automático, conocido informalmente como ¿Roca o Mina? Es una pequeña base de datos que consta de 111 patrones correspondientes a rocas y 97 a minas de agua (simuladas como cilindros metálicos). Cada uno de los patrones consiste en 60 medidas numéricas correspondientes a una sección de las secuencias de sonar. Estos valores ya están entre 0.0 y 1.0, aunque vale la pena normalizarlos para estar seguros. Estas medidas representan el valor de energía de diferentes rangos de longitudes de onda durante un cierto período de tiempo.

Vamos a utilizar un par de paquetes nuevos en el proceso, más específicamente, [DataFrames.jl](https://juliaai.github.io/DataScienceTutorials.jl/data/dataframe/) y [UrlDownload.jl](https://github.com/Arkoniak/UrlDownload.jl). Por lo tanto, lo primero es asegurarse de que los paquetes estén correctamente instalados.


In [ ]:
using Pkg;
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("UrlDownload")

Después de eso, los datos serán descargados si aún no están disponibles, para lo cual se puede utilizar el siguiente código:

In [ ]:
using UrlDownload
using DataFrames
using CSV

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
data = urldownload(url, true, format=:CSV, header=false) |> DataFrame
describe(data)

Como se puede ver en la línea anterior, hemos descargado los datos y los hemos canalizado, con el operador `|>`, en la función `DataFrame`. Esto creará una estructura similar a una tabla de base de datos, lo que resulta particularmente conveniente para verificar los valores faltantes o los rangos de las diferentes variables. De hecho, la biblioteca facilita especialmente el manejo de valores faltantes con funciones para completar o eliminar las muestras con medidas no válidas. Sin embargo, es demasiado extenso ver cada variable en el informe de salida; si se realizan algunas consultas, podemos identificar que no hay valores faltantes. Además, ninguna variable supera 1.0, aunque algunas de ellas no están normalizadas. Una estructura similar se puede encontrar en otros lenguajes, como R o Python.

Como ejemplo de este proceso, vamos a agregar una columna adicional para convertir a categórica la última columna, la columna 60, que tiene una **M** para cada Mina y una **R** para cada muestra de roca.

In [ ]:
insertcols!(data, :Mine => data[:, 61].=="M")

Una vez que los datos están cargados en el DataFrame para fines de verificación y se han aplicado los posibles procesos a los datos, como en los tutoriales anteriores, los datos deben ser puestos en forma de matriz, tal como:

In [ ]:
input_data = Matrix(data[!, 1:60]);
output_data = data[!, :Mine];

@assert input_data isa Matrix
@assert output_data isa BitVector

Cabe mencionar que en un DataFrame, cuando se consulta un conjunto de filas, como en el caso de `X`, el resultado también es un DataFrame. Por lo tanto, para aplicar las operaciones restantes, es necesario usar la función `Matrix` para obtener una matriz en la que se puedan utilizar las operaciones anteriores como de costumbre.


### Pregunta

Ahora, los datos están cargados y convertidos a los tipos habituales. A continuación, deberías poder aplicar en la siguiente sección la división del conjunto de datos en dos subconjuntos, de prueba y de entrenamiento, y aplicar la normalización correspondiente. Coloca el código en la siguiente sección para realizar ambas operaciones.

In [ ]:
#train_input, train_output, test_input, test_output = #TODO


## Línea Base

Como se mencionó anteriormente, los ensambles son un conjunto de clasificadores "más débiles" que nos permiten superar sus limitaciones al unirlos. Por eso, antes de comenzar con los ensambles, será necesario tener algunos modelos de referencia que luego se unirán en un meta-clasificador. En el siguiente ejemplo, se entrenan algunos modelos simples de la biblioteca `scikit-learn`: un SVM con kernel RBF, una Regresión Lineal, un Naïve Bayes y un Árbol de Decisión.


In [ ]:
using ScikitLearn

@sk_import svm:SVC
@sk_import tree:DecisionTreeClassifier
@sk_import linear_model:LogisticRegression
@sk_import naive_bayes:GaussianNB 

#Definir los modelos a entrenar
models = Dict( "SVM" => SVC(probability=true), 
         "LR" =>LogisticRegression(),
         "DT"=> DecisionTreeClassifier(max_depth=4),
         "NB"=> GaussianNB())

base_models =  [ name for name in keys(models)]

In [ ]:
# Perform the training for each model and calculate the test values (accuracy)
for key in keys(models)
    model = models[key]
    fit!(model,train_input, train_output)
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end

## Combinando modelos débiles en un ensamble

Cuando se trata de combinar los modelos, existen diferentes estrategias dependiendo de la tarea del modelo, es decir, si estamos clasificando o regresando. En este caso particular, nos vamos a enfocar en la clasificación, aunque para la regresión sería similar, pero se debe tener en cuenta la naturaleza continua de los valores al combinar las salidas.

En cuanto a la combinación de la clasificación, hay principalmente dos maneras de combinar las salidas de varios clasificadores. Estas combinaciones se llaman Votación Mayoritaria y Votación Mayoritaria Ponderada.

### Votación Mayoritaria
Aunque también conocida como Votación Dura, como su nombre indica, se basa en seleccionar la opción más votada entre las predicciones realizadas por los diferentes modelos. La implementación disponible en scikit-learn suma las predicciones para cada una de las clases y luego promedia estas estimaciones. La opción seleccionada por mayoría entre los "expertos" de los cuales el ensamble está compuesto es la elegida. De esta manera, el problema podría resolverse teniendo en cuenta diferentes resultados o puntos de vista sobre el problema. A continuación, se muestra un ejemplo en el código de cómo construir dicho modelo.

In [ ]:
@sk_import ensemble:VotingClassifier

#Define the metaclassifier based on the base_models
models["Ensemble (Hard Voting)"] = VotingClassifier(estimators = [(name,models[name]) for name in base_models], 
                                                   n_jobs=-1)
fit!(models["Ensemble (Hard Voting)"], train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end

Como se puede ver, aunque no mejora el rendimiento del mejor de los modelos componentes, esto se debe a que, en primer lugar, no se trata de un problema particularmente complejo. Además, otro problema es que confiamos igualmente en todos los modelos al decidir la clase de respuesta. Para resolver este problema, es posible hacer que no todos los modelos tengan la misma importancia, como veremos en la siguiente sección.

### Votación Mayoritaria Ponderada

Como se mencionó en la sección anterior, uno de los problemas del modelo clásico de *ensamble* es que todos los resultados tienen el mismo peso y en cada uno de los modelos "débiles" solo se tiene en cuenta la opción más votada. Para resolver esto, una de las propuestas es usar un peso en las decisiones. Esto se debe a que un modelo puede ser mejor que otro o más fiable. Para reflejar este punto, se puede modificar la salida multiplicándola por un factor de confianza dentro de la regla utilizada para tomar las decisiones. Este procedimiento de ponderación a veces también se conoce como *Votación Suave* en contraste con *Votación Dura* o votación no ponderada. Imagina que a cada uno de los clasificadores se le asigna el mismo peso, es decir, {1,1,1}. En un ejemplo como el siguiente, con un SVM, una regresión logística y un modelo basado en Bayes, obtendríamos las siguientes salidas:

| Clasificador     | Mina          | Roca          |
| :--------------- | :------------: | ------------: |
| SVM              | 0.9            | 0.1           |	
| LR               | 0.3            | 0.7           |	
| NB               | 0.2            | 0.8           |
| Votación Suave   | 0.47           | 0.63          |	

Por lo tanto, la clase seleccionada sería la clase Roca, ya que todos los modelos tienen el mismo peso en el proceso de toma de decisiones al promediar. En contraste, si sabemos que uno de los modelos es mejor, podemos ponderar la respuesta de ese modelo. Imagina en el ejemplo anterior si supieras que el SVM suele ser mucho mejor que los otros dos para este problema particular. En ese caso, puedes aumentar su peso como se muestra a continuación para tener en cuenta más a ese modelo. Con el mismo ejemplo, pero con la respuesta del SVM siendo mayor, los resultados serían:

| Clasificador     | Mina          | Roca          |
| :--------------- | :------------: | ------------: |
| SVM              | 2 * 0.9        | 2 * 0.1       |	
| LR               | 1 * 0.3        | 1 * 0.7       |	
| NB               | 1 * 0.2        | 1 * 0.8       |
| Votación Suave   | 0.575          | 0.425         |

Como se puede ver en los resultados, si tenemos un modelo de mayor calidad, las salidas de este modelo se tendrán más en cuenta para tomar la decisión correspondiente.

Para implementar este tipo de comportamiento, simplemente puedes agregar dos parámetros adicionales a la función `VotingClassifier` que se utilizó anteriormente para ponderar la salida.


In [ ]:
models["Ensemble (Soft Voting)"] = VotingClassifier(estimators = [(name,models[name]) for name in base_models], 
                                                   n_jobs=-1, voting="soft",weights=[1,2,2,1])
fit!(models["Ensemble (Soft Voting)"],train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,train_input, train_output)
    println("$key: $(acc*100) %")
end

Como puedes ver, los resultados son mejores cuando combinas varios modelos que ofrecen buenos resultados. De hecho, este procedimiento es la base de otras técnicas como el *Bosque Aleatorio* que veremos un poco más adelante en este tutorial. Los modelos a utilizar son la otra clave para la creación del _ensamble_. En la siguiente sección, veremos las estrategias más comunes para la creación de los modelos.

El ajuste de estos pesos se puede hacer de muchas maneras diferentes; por ejemplo, se puede hacer manualmente como lo hemos hecho en el ejemplo anterior. Otra alternativa sería utilizar una técnica de descenso de gradiente para ajustarlos, como si fuera una red neuronal o un SVM. Otra posibilidad es usar el valor de ajuste en el conjunto de validación (en este caso no se ha reservado un conjunto de datos para este propósito) como el peso de los modelos.

### Pregunta
Hemos realizado cada prueba con una estrategia de separación, sin embargo, como se indicó en una sesión anterior, la aplicación de un enfoque de validación cruzada es preferible para cortar la dependencia en la selección de las muestras. En este caso, podrías pensar que hay dos enfoques diferentes: uno es aplicar la validación cruzada a cada modelo, elegir el mejor y combinar esos en un solo ensamble. La otra opción sería aplicar la validación cruzada a nivel de ensamble antes de entrenar los modelos. ¿Cuál es el correcto y por qué?

`Responda aquí`

### Stacking

Este último enfoque para combinar los modelos puede considerarse una variante de la Votación Suave. Como se mencionó en esa sección, la votación suave permite que los pesos de cada uno de los modelos estén fijos, y estos pueden ajustarse con una técnica de gradiente decreciente. El *stacking* se identifica comúnmente como la creación de una técnica de clasificación superior a una regresión lineal (que es lo que hace la votación suave), como una red neuronal artificial (ANN) para combinar los modelos.

Así, como se ha hecho anteriormente, las salidas de las diferentes técnicas podrían tomarse y usarse como entradas para otro modelo de clasificación, permitiendo el ajuste de los pesos y las combinaciones no lineales de las respuestas de cada uno.

Puedes ver un ejemplo de esto en el siguiente código, que utiliza la implementación en `scikit-learn` que usa un SVC como modelo de combinación:


In [ ]:
@sk_import ensemble:StackingClassifier

models["Ensemble (Stacking)"] = StackingClassifier(estimators=[(name,models[name]) for name in base_models],
    final_estimator=SVC(probability=true), n_jobs=-1)
fit!(models["Ensemble (Stacking)"], train_input, train_output)

In [ ]:
for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end

## Creación de Modelos

Uno de los elementos clave que aún no se ha abordado es la creación de los modelos que compondrán el meta-clasificador. Hasta ahora, el enfoque seguido no es muy adecuado, ya que el conjunto de datos de entrada para todos los modelos es el mismo. Esto tiene el efecto de una falta obvia de diversidad en los modelos, ya que cualquiera que sea el modelo que creemos, tendrá la misma información o "punto de vista" que los demás. Sin embargo, esta no es la práctica habitual. En su lugar, el conjunto de patrones de entrada se divide generalmente en conjuntos más pequeños con los que entrenar una o más técnicas, con el fin de reducir el costo computacional por un lado y aumentar la diversidad de los modelos por otro. Es necesario recordar en este punto que los modelos "débiles" no tienen que ser perfectos en todas las clases e incluso no tienen que cubrir todas las posibilidades, solo modelos que sean rápidos de entrenar y ofrezcan una salida más o menos consistente.

En cuanto a la forma en que se debe particionar los datos para la creación de los modelos, la mayoría de los enfoques consideran dos enfoques principales conocidos como *Bagging* y *Boosting*. A continuación, se describirán brevemente estos dos enfoques.

### Bagging o agregación bootstrap

La técnica conocida como _Bagging_ o selección con reemplazo fue propuesta por Breiman en 1996. Se basa en el desarrollo de múltiples modelos que pueden entrenarse en paralelo. El elemento clave de estos modelos es que cada modelo se entrena en un subconjunto del conjunto de entrenamiento. Este subconjunto de datos se extrae de manera aleatoria con reemplazo. Este último punto es particularmente importante porque, una vez que se ha seleccionado un ejemplo de las posibilidades, se coloca de nuevo entre las posibilidades para que pueda ser seleccionado ya sea en el subconjunto que se está construyendo, o en los subconjuntos de los otros modelos, es decir, se crean conjuntos de ejemplos no disjuntos.

![Ejemplo de Bagging](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Ensemble_Bagging.svg/440px-Ensemble_Bagging.svg.png)

El resultado es que se crean "expertos" especializados en datos y dependiendo de la partición. Mientras que los datos comunes, o más frecuentes, están correctamente cubiertos por todos los modelos, también es cierto que los datos menos frecuentes tienden a no estar en todas las particiones y pueden no estar cubiertos en todos los casos. Así, se obtendrían modelos que estarían más especializados en ciertos datos o tendrían un punto de vista diferente, siendo expertos en una región particular del espacio de búsqueda.

Aunque se discutirá en más detalle más adelante, una técnica bien conocida que utiliza este enfoque para la construcción de sus modelos "débiles" es el RandomForest. Construye los árboles de decisión que componen el meta-clasificador de esta manera. Cualquier clasificador puede usarse como base de un *Bagging* con la clase [BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html).

Por ejemplo, en el siguiente código, se han elegido 10 SVM para clasificación como modelos débiles. Cada uno de esos modelos ha sido entrenado solo con el 50% de los patrones de entrenamiento, y por lo tanto, se debería aumentar la varianza entre ellos. 

In [ ]:
@sk_import svm:SVC
@sk_import ensemble:BaggingClassifier

models["Bagging (SVC)"] = BaggingClassifier(base_estimator=SVC(),n_estimators=10, max_samples=0.50, n_jobs=-1)
fit!(models["Bagging (SVC)"], train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end

Como alternativa a la extracción de ejemplos completos, se podría realizar una partición vertical del conjunto de entrenamiento, extrayendo así características. Para implementar esta alternativa, en la función `BaggingClassifier` se debe definir el parámetro *max_features*. Este enfoque se utiliza cuando el número de características es particularmente alto para crear modelos más simples que no utilicen toda la información, que a menudo es redundante. Cabe destacar que este procedimiento de extracción de características para modelos se realiza sin reemplazo, es decir, las características extraídas para un clasificador no se reintroducen en la lista de posibilidades hasta que se crea el conjunto para el siguiente clasificador.

### Boosting

La otra gran familia de técnicas para el meta-modelado de ensambles es lo que se conoce como *Boosting*. En este caso, el enfoque es ligeramente diferente, ya que el objetivo es crear una cadena de clasificadores. El elemento clave de este tipo de clasificador es que cada nuevo clasificador esté más especializado en los patrones que los modelos anteriores han pasado por alto. Por lo tanto, al igual que en el caso anterior, se selecciona un subconjunto de patrones del conjunto original. Sin embargo, este proceso se realiza de manera secuencial y sin reemplazo. Este punto es crucial ya que, como se mencionó anteriormente, la idea es eliminar aquellos patrones que ya están correctamente clasificados y obtener modelos más específicos que se concentren en aquellos ejemplos que son menos frecuentes o que han sido clasificados incorrectamente en un paso anterior. Así, al igual que en *Bagging*, la idea subyacente de este enfoque es que no todos los modelos deben tener todos los patrones como base, pero a diferencia de _Bagging_, este proceso es lineal debido a la dependencia en la construcción de los modelos.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Ensemble_Boosting.svg/1920px-Ensemble_Boosting.svg.png" alt="Ejemplos de Boosting" width="600"/>

Posteriormente, para obtener la combinación de los modelos, se utiliza el Majority Vote con pesos. En este enfoque, los pesos se establecen con un sistema de aproximación iterativa. Existen muchos ejemplos que utilizan este tipo de técnica, como [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) o [Gradient Tree Boosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html). En ambos casos, lo que se hace es un ajuste de los pesos con una técnica basada en el Descenso de Gradiente.

En el caso de AdaBoost, el algoritmo comienza dando un peso a todas las instancias del conjunto de entrenamiento. Con este conjunto ponderado, se entrena un clasificador con los datos originales. Dependiendo de los errores cometidos, se ajustan los pesos del conjunto original y se entrena una nueva copia del clasificador, pero con los datos ajustados, que se centrarán más en las instancias que se han clasificado incorrectamente. En el caso de `scikit-learn`, el algoritmo implementado se conoce como [AdaBoost-SAMME](https://hastie.su.domains/Papers/SII-2-3-A8-Zhu.pdf) propuesto por Zhu et al. en 2009. Como particularidad de esta implementación, la función de *pérdida* utilizada es una exponencial. Esta es la que se usará para calcular el peso de los errores cometidos, así como el peso de los clasificadores en el meta-clasificador. En términos generales, la salida será la más votada por los clasificadores en función del peso de cada uno de ellos.

El Gradient Tree Boosting es un enfoque diferente al uso de Boosting. Construye un árbol donde los nodos del árbol establecen los criterios para, por ejemplo, en el caso de clasificación, referirse a la `logistic-likelihood` de un patrón dado. De esta manera, cada uno de los nodos del árbol hace una clasificación que se ajusta en función de los errores residuales que se producen al ajustar los pesos de los diferentes clasificadores en el árbol. Esta división se lleva a cabo para cada una de las características disponibles, realizando un procedimiento recursivo al entrenar varios clasificadores de esta manera. Posteriormente, para tomar la decisión, se basa en las respuestas de los clasificadores por los que ha pasado. La principal diferencia con AdaBoost es que en este caso la salida son las probabilidades de las clases, que se suman para dar la respuesta más probable en lugar de la respuesta sobre las instancias.

A continuación, se muestra un enfoque con un ejemplo de uso de estos dos meta-clasificadores que hacen uso de _Boosting_.

In [ ]:
@sk_import ensemble:(AdaBoostClassifier, GradientBoostingClassifier)

models["Ada"] = AdaBoostClassifier(n_estimators=30)
fit!(models["Ada"], train_input, train_output)

models["GTB"] = GradientBoostingClassifier(n_estimators=30, learning_rate=1.0, max_depth=2, random_state=0)
fit!(models["GTB"], train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end


### Pregunta
De manera similar a lo realizado en la sección de validación cruzada, desarrolla una función para entrenar ensamblados. La función, denominada trainClassEnsemble, también seguiría una validación cruzada estratificada. A continuación, un recordatorio rápido de los pasos que debe cubrir la función:

1. Crear un vector con $k$ elementos, que contendrá los resultados de prueba del proceso de validación cruzada utilizando la métrica seleccionada.

2. Realizar un bucle con $k$ iteraciones (k particiones) en el que, dentro de cada iteración, a partir de las matrices de entradas y salidas deseadas, y mediante el vector de índices resultante de la función anterior, se crean 4 matrices: entradas y salidas deseadas para entrenamiento y prueba.

3. Dentro de este mismo bucle, añadir una llamada para generar los modelos, que pueden ser cualquiera de los utilizados anteriormente.

4. Entrenar dichos modelos usando el conjunto de entrenamiento correspondiente, es decir, los $k−1$ subconjuntos restantes no utilizados para prueba.

5. En caso de necesitar un conjunto de validación, por ejemplo, $\omega$, dividir el conjunto de entrenamiento en dos partes. Para esto, usa un *holdOut*.

6. Construir el ensamblado siguiendo alguna de las estrategias descritas anteriormente (cualquiera de ellas) y calcular el resultado de prueba.

Finalmente, proporcionar el resultado de promediar los valores de estos vectores para cada métrica, junto con sus desviaciones estándar.

Como resultado de esta función, se debería devolver al menos el valor de prueba en la(s) métrica(s) seleccionada(s).


In [ ]:
function trainClassEnsemble(estimators::AbstractArray{Symbol,1}, 
        modelsHyperParameters:: AbstractArray{Dict, 1},     
        trainingDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}},    
        kFoldIndices::     Array{Int64,1})
    #TODO
end

### Pregunta
Repite la función anterior, pero esta vez permitiendo pasar solo un estimador como base. Este puede ser replicado y luego pasado a la función anterior.

In [ ]:
function trainClassEnsemble(baseEstimator::Symbol, 
        modelsHyperParameters::Dict,
        NumEstimators::Int=100,
        trainingDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}},     
        kFoldIndices::     Array{Int64,1})
    #TODO
end

## Técnicas que integran el enfoque de Ensemble

Algunos de los algoritmos más conocidos y actualmente utilizados se basan en este tipo de enfoque. Entre estos enfoques, quizás los más famosos y ampliamente utilizados son aquellos basados en la generación de árboles de decisión simples (DT). La razón del uso de los árboles es su fácil interpretación, así como la velocidad de cálculo y entrenamiento. A continuación, veremos los dos enfoques conocidos hoy en día en este sentido: ***Random Forest*** y ***XGBoost***.

### Random Forest

Este algoritmo, propuesto por Breitman y Cutler en 2006 basado en una publicación anterior de Ho en 1995 (_Random Subspaces_), es la técnica de ensamble paradigmática. El algoritmo une en un ensamble un conjunto de clasificadores simples que toman la forma de árboles de decisión. Estos clasificadores se entrenan siguiendo un enfoque de **bagging**, y por lo tanto, pueden ser entrenados en paralelo. La combinación de las salidas de los algoritmos se realiza para problemas de clasificación mediante la opción más votada entre los "expertos" o, si se trata de un problema de regresión, mediante la media aritmética de las respuestas.

Es un algoritmo que necesita el ajuste de muy pocos hiperparámetros para obtener muy buenos resultados en casi cualquier tipo de problema. En general, el valor más importante es el número de estimadores y, por lo tanto, el número de particiones que se deben realizar del conjunto de entrenamiento. Varios autores señalan que este número de estimadores debería ser $\sqrt{\textrm{Número de variables}}$ para problemas de clasificación, y $\frac{\textrm{Número de variables}}{3}$ para problemas de regresión. Sin embargo, también se señala que la técnica se saturaría entre 500 y 1000 árboles y, por más que se aumente, no mejoraría los resultados. Sin embargo, este último punto solo se ha probado empíricamente en ciertos conjuntos de datos y, por lo tanto, debe tomarse con cautela ya que no tiene justificación matemática.

Además del proceso de bagging habitual, el Random Forest también incluye un segundo mecanismo de partición. Una vez seleccionados los patrones que formarán parte del conjunto de entrenamiento del árbol de decisión, solo un subconjunto de características (*features*) está disponible para cada nodo del árbol. Esto aumenta la diversidad de los árboles en el bosque y se centra en el rendimiento general con una pequeña varianza en los resultados. Este mecanismo permite evaluar cuantitativamente el rendimiento individual de cada árbol en el bosque y sus variables. Por lo tanto, se puede medir la importancia de cada variable. Esta medida que calibra la participación de cada variable en los nodos del árbol en la toma de decisiones se llama impureza y mide la diferencia entre las diferentes ramas del árbol al particionar los ejemplos. A veces, esta misma medida se utiliza como una medida para la selección de variables tomando la medida en todos los árboles del bosque de la participación e importancia mediante un filtrado como los vistos en la unidad anterior.

Para el cálculo de esta medida de impureza, existen diferentes enfoques. Por ejemplo, `scikit-learn` utiliza una medida que llama **Gini**. Esta última es la probabilidad de clasificar erróneamente un elemento elegido al azar en el conjunto de datos si fuera etiquetado aleatoriamente de acuerdo con la distribución de clases en el conjunto de datos. Se calcula como:
$$G = \sum_{i=1}^C p(i) * (1 - p(i))$$

donde $C$ es el número de clases y $p(i)$ es la probabilidad de seleccionar aleatoriamente un elemento de la clase $i$. Un buen ejemplo de cómo calcular la impureza de las ramas se puede ver en el siguiente [enlace](https://victorzhou.com/blog/gini-impurity/).

A continuación, en el ejemplo utilizado en esta unidad, ejecutaremos un modelo de *Random Forest* con la implementación de `scikit-learn`. Los parámetros más importantes de esta implementación son:

- ***n_estimators***, que marca el número de árboles a generar o el número de particiones de *bagging*.
- ***criterion***, medida de impureza de nodo. Por defecto se usa Gini, pero se puede cambiar a ganancia de entropía.
- ***max_depth***, permite limitar la profundidad máxima de los árboles para limitar el número de variables a usar.
- ***min_samples_split***, para cada árbol de decisión, cuántos patrones se necesitan para realizar una división interna en los *Decision Trees*.
- ***bootstrap***, se puede usar el enfoque de *bagging* o *bootstrap* para construir los árboles, pero si esta propiedad es falsa, entonces utiliza todo el conjunto de entrenamiento para generar los árboles. En caso de un valor True, se tienen en cuenta las siguientes propiedades:
    + ***max_samples***, número de ejemplos a extraer del conjunto original para construir el conjunto de entrenamiento del estimador, el valor por defecto es igual al número de patrones pero recuerda que el mismo puede ser extraído varias veces ya que es una selección con reemplazo dando variabilidad.
    + ***oob_score***, medida *out of bag* para estimar la generalización. Aquellas muestras que no han formado parte del entrenamiento de un estimador pueden ser usadas para calcular una medida de validación, y promediadas entre todos los estimadores para ver cuán general es el bosque construido.
    
Por ejemplo, el código a continuación muestra cómo usar la implementación en `scikit-learn`:

In [ ]:
@sk_import ensemble:RandomForestClassifier

models["RF"] = RandomForestClassifier(n_estimators=8, max_depth=nothing,
                                    min_samples_split=2, n_jobs=-1)
fit!(models["RF"], train_input, train_output)
    
for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end

En este enfoque, el número de estimadores se ha definido siguiendo la regla mencionada de $\sqrt{\textrm{Número de variables}}$. En este caso, dado que hay pocos estimadores y pocos patrones, los resultados pueden variar bastante dependiendo del tipo de particiones obtenidas.

Luego, una vez que el modelo ha sido entrenado, el nivel de impureza obtenido para cada una de las frecuencias calculadas con el algoritmo Gini puede ser calculado como un promedio de aquellos obtenidos entre los árboles que componen el bosque. Esto se puede realizar de la siguiente manera en `scikit-learn`: 

In [ ]:
p = bar(y=1:60,models["RF"].feature_importances_, orientation=:horizontal, legend = false)
xlabel!(p,"Gini Gain")
ylabel!(p,"Fearure")
title!("Feature Importance")

Debe señalarse que, como se puede observar en el gráfico, este valor determina que la mayor parte de la información está concentrada en algunas de las frecuencias utilizadas. Por esta razón, podría llevarse a cabo un filtrado de la información, como los que veremos en la siguiente sesión, basado en este valor.

### XGBoost (eXtreme Gradient Boosting)

Finalmente, en esta última sección, se debe mencionar nuevamente el **Gradient Boosting**, específicamente una implementación que en los últimos años se ha vuelto muy famosa por su versatilidad y rapidez. Esta implementación se conoce como ***XGBoost (eXtreme Gradient Boosting)***, que se ha destacado especialmente en competiciones como la plataforma Kaggle por su rapidez en obtener resultados y robustez.

***XGBoost*** será un ensemble similar al Random Forest. Este cambio proviene de la necesidad de que el algoritmo obtenga la probabilidad de las decisiones, como fue el caso con *Gradient Tree Boosting*. El otro cambio fundamental en este algoritmo, dado que se basa en *Gradient Tree Boosting*, es el cambio de la estrategia de *bagging* a la de *boosting* para la creación de los conjuntos de entrenamiento del clasificador.

Posteriormente, esta técnica realiza un enfoque de entrenamiento aditivo cuyos pesos se ajustan en base a un **Gradiente Decreciente** sobre una función de **pérdida** a definir. Al añadir la función de pérdida con el término de regularización, se puede calcular la segunda derivada de las funciones para actualizar los pesos de clasificación de los diferentes árboles. El cálculo de este gradiente permite ajustar los valores de los clasificadores que se generan siguiendo uno dado para permitir que los pesos concentren la atención en los patrones que están mal clasificados. Los detalles matemáticos de la implementación se pueden encontrar en este [enlace](https://xgboost.readthedocs.io/en/stable/tutorials/model.html).

A diferencia de los otros enfoques que hemos visto, `xgboost` no está actualmente implementado en `scikit-learn`. Por esta razón, se debe instalar la versión de referencia si no está ya presente en la máquina.

In [ ]:
using Pkg;
Pkg.add("XGBoost")

Después de la instalación, la biblioteca se puede usar como se muestra en el siguiente ejemplo. A diferencia de otras implementaciones, la implementación en Julia admite como entrada Arrays de Julia, SparseMatrixCSC, texto en formato libSVM y archivos binarios de XGBoost. Aunque la biblioteca de Julia ofrece muchas opciones para cambiar internamente al formato [LIBSVM](https://xgboost.readthedocs.io/en/stable/tutorials/input_format.html), como cualquier otra biblioteca, esta implementación no tiene todas las posibilidades y, en particular, el BitVector no está soportado actualmente en su función `DMatrix`. Por lo tanto, se requiere un pequeño cambio en el formato para usar la biblioteca. En las últimas versiones es posible que esto no sea nacesario ya que las llamadas a **XGBoost** permiten el uso de vectores e internamente hará el cambio, si bien el paso de las DMatrix es más eficiente.

In [ ]:
using XGBoost;

train_input = input_data
train_output = output_data

test_input = input_data
test_output = output_data

train_output_asNumber= Vector{Number}(train_output);

@assert train_output_asNumber isa Vector{Number}

Una vez realizada esta adaptación de datos, puedes proceder con el entrenamiento de un modelo utilizando la biblioteca `xgboost`. Para hacerlo, solo es necesario llamar a la función `train` con los parámetros correspondientes. Entre estos parámetros, los más importantes son:

- **eta**, término que determinará la compresión de los pesos después de cada nueva etapa de *boosting*. Toma valores entre 0 y 1.
- **max_depth**, profundidad máxima de los árboles, que por defecto tiene un valor de 6. Aumentarlo permitirá modelos más complejos.
- **gamma**, parámetro que controla la reducción mínima de la pérdida necesaria para realizar una nueva partición en un nodo hoja del árbol. Cuanto mayor sea, más conservador será.
- **alpha** y **lambda**, son los parámetros que controlan la regulación L1 y L2 respectivamente.
- **objective**, define la función de pérdida que se utilizará, la cual puede ser una de las predefinidas, que se pueden consultar en este [enlace](https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster).

Luego, solo es necesario establecer el número máximo de iteraciones del proceso de *boosting*, como se muestra en el siguiente ejemplo con 20 rondas.

In [ ]:
svm_data = DMatrix(train_input, label=train_output_asNumber)

model = xgboost(svm_data, rounds=20, eta = 1, max_depth = 6)

En el siguiente fragmento de código, varios parámetros se pasan como un diccionario y se calculan dos métricas diferentes. La primera métrica, **error**, se refiere a las muestras clasificadas incorrectamente sobre el total de muestras. La segunda métrica es el Área Bajo la Curva ROC (**AUC**).

In [ ]:
param = ["max_depth" => 2,
         "eta" => 1,
         "objective" => "binary:logistic"]
metrics = metrics = ["error", "auc"]
model = xgboost(DMatrix(train_input, label=train_output_asNumber), rounds=20, param=param, metrics=metrics)

pred = predict(model, train_input)

***Importante***.

En caso de que se utilice un conjunto de validación, este debe pasarse en el parámetro *evals* de la función de entrenamiento. Además, y solo cuando se defina el parámetro *evals*, puedes establecer el número de rondas para el pre-stop con el parámetro *early_stopping_rounds* de la función de entrenamiento. El código sería similar a:

``` julia
    evals = DMatrix(val_input, label=val_output)
    xgb_model = xgb.train(param, train_input, num_round,label = train_output_asNumber, evals=evals,
                    early_stopping_rounds=10)
```

El valor proporcionado en la salida corresponde a la suma de las salidas de los árboles, estando entre 0 y 1 para la pertenencia a una clase determinada. Dado que se trata de una clase binaria, simplemente establece un umbral de 0.5 en la salida para determinar cuál es la respuesta.

In [ ]:
using XGBoost: predict as predict_xgb

pred = predict_xgb(model, test_input)
print("Error of XGboost= ", sum((pred .> 0.5) .!= test_output) / float(size(pred)[1]), "\n")

Finalmente, al igual que en el caso del Random Forest, es posible identificar la importancia y representarla gráficamente para cada una de las variables en el ranking. Con el siguiente código es posible ver dicho marcador ordenado de manera ascendente:


In [ ]:
feature_gain =  [(first(x),last(x)) for x in importance(model)]
feature, gain = first.(feature_gain), last.(feature_gain)

using Plots;

p = bar(feature, y=gain, orientation="h", legend=false)
xlabel!(p,"Gain")
ylabel!(p,"Feature")
title!("Feature Importance")

Como puede verse, no todas las características tienen la misma importancia. Cabe destacar que el eje de características identifica la posición en el vector de características, que está ordenado por el valor de ganancia por defecto.

### LIGHTGBM
Al igual que ocurre con el caso de **XGBoost** deberemos instalar una librería que descargará la ejecución en Python, por lo que se previene de que esta parte del código puede ser particularmente lenta. Como se destaca en la clase de teoría, LightGBM es un sistema que está pensado principalmente para trabajar con grandes volumenes de datos, para lo que intenta solucionar algunos de los problemas asociados a **XGBoost**. 


#### Instalación
Para uso, los pasos serán en primer lugar proceder con la instalación:

In [ ]:
using Pkg;
Pkg.add("LightGBM")

A partir de ese punto se procederá al uso y configuración de los parámetros de la librería 

In [ ]:
using LightGBM;

train_input = input_data
train_output = output_data

test_input = input_data
test_output = output_data

model = LGBMClassification(
    objective = "binary",
    num_iterations = 100,
    learning_rate = .1,
    early_stopping_round = 5,
    feature_fraction = .8,
    bagging_fraction = .9,
    bagging_freq = 1,
    num_leaves = 1000,
    num_class = 1,
    metric = ["auc", "binary_logloss"]
)


Como se pude ver, los parámetros más importantes son:

* **objective: "binary"**. Define la función objetivo que el modelo intentará optimizar. En este caso, "binary" indica que es un problema de clasificación binaria.
* **num_iterations: 100.**  El número total de iteraciones (o árboles) que se construirán durante el entrenamiento. Un número mayor puede mejorar la precisión, pero también puede llevar a un sobreajuste.
* **learning_rate: .1**. La tasa de aprendizaje, también conocida como "eta". Controla cuánto ajusta el modelo los pesos en cada iteración. Un valor menor puede hacer que el modelo aprenda más lentamente pero de manera más precisa.
* **early_stopping_round: 5**. Si se usa un conjunto de validación, el entrenamiento se detendrá si no hay mejora en la métrica de evaluación durante early_stopping_round iteraciones consecutivas. Esto ayuda a evitar el sobreajuste.
* **feature_fraction: .8**. El porcentaje de características (features) que se usan en cada iteración para construir un árbol. Reducir este valor puede ayudar a evitar el sobreajuste y acelerar el entrenamiento.
* **bagging_fraction: .9**. El porcentaje de datos que se utilizan en cada iteración para construir los árboles. Es una forma de realizar "bagging" y también ayuda a evitar el sobreajuste.
* **bagging_freq: 1**. La frecuencia con la que se realiza el bagging. Si se establece en 1, el bagging se realiza en cada iteración. Si se establece en un valor mayor, el bagging se realiza cada bagging_freq iteraciones.
* **num_leaves: 1000**. El número máximo de hojas en un árbol. Aumentar este valor puede permitir que el modelo capture más complejidad, pero también puede llevar a un sobreajuste.
* **num_class: 1**. El número de clases en el problema de clasificación. Para clasificación binaria, se establece en 1. Para clasificación multiclase, se establece en el número de clases.
* **metric: ["auc", "binary_logloss"]**. La lista de métricas que se utilizarán para evaluar el rendimiento del modelo. "auc" es el Área Bajo la Curva ROC, y "binary_logloss" es la pérdida logarítmica binaria, que mide la calidad de las predicciones.

Lo único que queda es entrenar y usar el modelo

In [ ]:
# Fit the estimator on the training data and return its scores for the test data.
fit!(model, train_input, train_output, (test_input, test_output))

# Predict arbitrary data with the estimator.
predict(model, test_input)

Un punto que puede ser interesante en algun caso, es que el modelo peude ser guardado y cargado de manera sencilla con las siguientes líneas de código:

```julia
# Save and load the fitted model.
filename = pwd() * "/lightGBM.model"
savemodel(model, filename)
loadmodel!(model, filename)

```

Otros elementos que incluye la librería son las fucniones `search_cv` o `cv` con las que hacer una busqueda de cross-validation o de GridSearchCV.


### CatBoost

CatBoost es un algoritmo de machine learning basado en árboles de decisión, desarrollado por Yandex. Su nombre proviene de "Categorical Boosting", ya que es particularmente eficaz en la gestión de características categóricas sin necesidad de un preprocesamiento extensivo. CatBoost es una de las implementaciones modernas de técnicas de ensemble, junto con LightGBM y XGBoost.

#### Instalación
Para comenzar a usar CatBoost, es necesario instalar la biblioteca en Python. A continuación, se muestra cómo instalar CatBoost en Python y cómo usar su interfaz en Julia.

In [ ]:
!pip install catboost

In [ ]:
Pkg.add("CatBoost")

Desde ese punto se puede hacer un uso similar al de LighGBM o XGboost, soportando varios formatos.

In [ ]:
using CatBoost;
using PythonCall;

train_input = input_data
train_output = output_data

test_input = input_data
test_output = output_data

model = CatBoostclassifier(iterations = 2, learning_rate = 1, depth = 2)

La función `CatBoostClassifier` cuenta con los siguientes parámetros como más importantes:

 * **iterations: 1000** Número de iteraciones (o árboles) que se entrenarán en el modelo.
 * **learning_rate: 0.03** Tasa de aprendizaje del modelo.
 * **depth: 6** Profundidad máxima de los árboles individuales.
 * **l2_leaf_reg: 3.0** Coeficiente de regularización L2.
 * **random_seed** Semilla para la generación de números aleatorios.
 * **logging_level: "Verbose"** Controla la verbosidad durante el entrenamiento. Se puede configurar como "Silent" para desactivar la salida durante el entrenamiento o "Verbose" para ver detalles del progreso.
 * **eval_metric:** Métrica utilizada para evaluar la calidad del modelo.
 * **early_stopping_rounds** Número de iteraciones sin mejora después de las cuales el entrenamiento se detendrá.
 * **one_hot_max_size: 2** Límite superior en el número de categorías únicas para las cuales se aplicará one-hot encoding.
 * **bootstrap_type: "Bayesian"** Tipo de método de bootstrap a utilizar.Opciones incluyen "Bayesian", "Bernoulli", "Poisson", etc.
 * **class_weights** Ponderaciones para cada clase.
 * **task_type: "CPU"** Define si el entrenamiento se realiza en CPU o GPU.

Una vez definido el modelo este puede entrenarse y usarse como sigue

In [ ]:
# Fit model
fit!(model, train_data, train_labels)

# Get predictions
preds = predict(model, eval_data)